In [8]:
# # install particular dependencies
# !pip install git+https://github.com/pangeo-forge/pangeo-forge-runner.git #pangeo-forge-runner from main


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.1.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['apache-beam']

conda-forge/linux-64                                        Using cache
conda-forge

In [2]:
# export keys
!export GOOGLE_APPLICATION_CREDENTIALS=/home/jovyan/KEYS/leap-pangeo-dataflow_service_account.json

In [3]:
!pangeo-forge-runner --help

/bin/bash: line 1: pangeo-forge-runner: command not found


In [2]:
# input args
repo = "https://github.com/jbusecke/cmip6-leap-feedstock.git"
ref = "beam-dataflow-testing"
f_name = "config.json"
target_bucket = "gs://leap-persistent/jbusecke/data/CMIP6/dataflow_test/target"

In [3]:
def shorten_id(iid):
    iid_stripped = "-".join(iid.split(".")[3:])
    assert len(iid_stripped) < 66
    return iid_stripped

In [4]:
# from the integration tests of pangeo-forge-runner
# https://github.com/pangeo-forge/pangeo-forge-runner/blob/main/tests/integration/test_dataflow_integration.py

import subprocess
import time
import xarray as xr

iid = "CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r107i1p1f1.day.psl.gr.v20200412"
jobname = shorten_id(iid)
target_store = f"{target_bucket}/{iid}.zarr"
machine_type = "e2-highmem-8"
cmd = [
    "pangeo-forge-runner",
    "bake",
    "--repo",
    repo,
    "--ref",
    ref,
    "--json",
    "-f",
    f_name,
    "--Bake.recipe_id",
    iid,
    "--Bake.job_name",
    jobname,
    "--TargetStorage.root_path",
    f"{target_bucket}",
    "--DataflowBakery.machine_type",
    machine_type,
]

print("\nSubmitting job...")
submit_proc = subprocess.run(cmd, capture_output=True)
assert submit_proc.returncode == 0
lastline = json.loads(submit_proc.stdout.decode().splitlines()[-1])
assert lastline["status"] == "submitted"
job_id = lastline["job_id"]
job_name = lastline["job_name"]
print(f"Job submitted with {job_id = }")


Submitting job...


FileNotFoundError: [Errno 2] No such file or directory: 'pangeo-forge-runner'

In [ ]:
# note the start time, because certain errors on dataflow manifest as long hangs,
# and if that's the case, we'll want to bail out of this test manually, rather than
# wait for the the job to officially fail.
start = time.time()

# 6 minutes seems like an average runtime for these jobs, but being optimistic
# let's start by waiting 5 minutes
print(f"Waiting for 5 mins, starting at {start = }")
time.sleep(60 * 5)

# okay, time to start checking if the job is done
show_job = f"gcloud dataflow jobs show {job_id} --format=json".split()
while True:
    elapsed = time.time() - start
    print(f"Time {elapsed = }")
    if elapsed > 60 * 12:
        # if 12 minutes have elapsed (twice the expected time to complete the job),
        # we're going to assume the job is hanging, and call this test a failure.
        # remember: we're sourcing data for this job from within GCS, so networking
        # shouldn't delay things *too* much. if we eventually find that jobs may take
        # more than 12 minutes and not be hanging, we can change this assumption.
        pytest.fail(f"Time {elapsed = } exceedes 12 minutes.")

    # check job state
    state_proc = subprocess.run(show_job, capture_output=True)
    assert state_proc.returncode == 0
    state = json.loads(state_proc.stdout)["state"]
    print(f"Current {state = }")
    if state == "Done":
        # on Dataflow, "Done" means success
        break
    elif state == "Running":
        # still running, let's give it another 30s then check again
        time.sleep(30)
    else:
        # consider any other state a failure
        pytest.fail(f"{state = } is neither 'Done' nor 'Running'")

# open the generated dataset with xarray!
gpcp = xr.open_dataset(
    config["TargetStorage"]["root_path"].format(job_name=job_name),
    engine="zarr",
)

assert (
    gpcp.title
    == "Global Precipitation Climatatology Project (GPCP) Climate Data Record (CDR), Daily V1.3"
)
# --prune prunes to two time steps by default, so we expect 2 items here
assert len(gpcp.precip) == 2
print(gpcp)